# CVX 2021-2022 Project 5 

In [1]:
# Import packages.
# free SDP solver: https://github.com/TrishGillett/pysdpt3glue
import cvxpy as cp
import numpy as np
from numpy.linalg import matrix_power
from numpy import linalg as LA
# make sure to use np.zeros() when you need something to initialize at 0. 

In [2]:
# The given data contains for E trajectories (different initial conditions) the vector x_{K+1}.
# load that data
data = np.loadtxt("datax.txt")

# Define the dynamics
Adyn = np.matrix([[0.5, 0.8], [0, 0.5]])
n = 2
Q0 = np.matrix([[1, 0], [0, 1]])
Qw = np.matrix([[1, 0], [0, 1]])

In [3]:
data.shape

(2, 1000)

## Empirical probability

In [4]:
def compute_e(Q0, Qw, Adyn, k):
    E_p = np.zeros(Q0.shape)
    for i in range(k + 1):
        mat_temp = matrix_power(Adyn, (k-i))
        E_p += mat_temp @ Qw @ mat_temp.T
        
    return matrix_power(Adyn, k) @ Q0 @ (matrix_power(Adyn, k)).T + E_p

In [5]:
# Compute the first two moments 
K = 10
E_pk = np.zeros((2,1))
E_pkkT = compute_e(Q0, Qw, Adyn, K + 1)

In [6]:
# compute the empirical probability of being in the safe set 
E = 1000
saft_range = 8

real_norm = np.linalg.norm(data, axis = 0, ord=1)
emp_prob = (real_norm < saft_range).sum() / E
print('Empirical probability after {} iterations : {:.2f}%'.format(K, emp_prob*100))

Empirical probability after 10 iterations : 99.80%


## SDP Solution

In [7]:
b = 1 / 2 * np.array([[1.0, 1], [1, -1], [-1, 1], [-1, -1]])

c = np.array([-saft_range, -saft_range, -saft_range, -saft_range])

P = cp.Variable((n, n), symmetric=True)
q = cp.Variable((n, 1))
r = cp.Variable((1, 1))
m = 4
tau = cp.Variable((m, 1))


obj = 1 - cp.trace(E_pkkT @ P) - 2 * (q.T @ E_pk) - r

constr = []

for i in range(m):
    constr.append(
        cp.bmat(
            [[P, q - tau[i] * np.atleast_2d(b[i]).T],
             [(q - tau[i] * np.atleast_2d(b[i]).T).T, (r - 1 - tau[i] * c[i])]]
        ) >> 0
    )

constr.append(cp.bmat([[P, q], [q.T, r]]) >> 0)

constr.append(tau >= 0)

prob = cp.Problem(cp.Maximize(obj), constr)
prob.solve()

print("Derived probability after {} iterations : {:.2f}%".format(K, prob.value * 100))


Derived probability after 10 iterations : 86.39%
